# Linear NN for images


## Imports


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


## Constants


In [ ]:
IMG_HEIGTH = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3


CLASS_NAMES = ["display", "dandelion", "roses", "sunflowers", "tulips"]


## TO DO


1. Load the dataset
2. JPEG to rgb
3. scale to 0-1
4. resize to 244*244*3
5. sanity check


In [ ]:
def read_and_decode(filename, resize_dims):
    """
    1. read the image dataset
    2. decode image data
    3. convert pixel values to floats in [0, 1]
    4. Resize the image to match desired dimensions
    """
    img_bytes = tf.io.read_file(filename=filename)
    img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
    img = tf.image.convert_image_dtype(img, dtype=tf.float32)
    img = tf.image.resize(images=img, size=resize_dims)
    return img


def parse_csvline(csv_line):
    record_default = ["", ""]
    filename, label_string = tf.io.decode_csv(csv_line, record_default)
    img = read_and_decode(filename, [IMG_HEIGTH, IMG_WIDTH])
    label = tf.argmax(tf.math.equal(CLASS_NAMES, label_string))
    return img, label


In [ ]:
train_dataset = (
    tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photo/train_set.csv")
    .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(16)
    .prefetch(tf.data.AUTOTUNE)
)


eval_dataset = (
    tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
    .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(16)
    .prefetch(tf.data.AUTOTUNE)
)


2025-08-03 22:46:19.063001: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-08-03 22:46:19.063761: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-08-03 22:46:19.064472: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-08-03 22:46:19.064871: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-08-03 22:46:19.065314: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
for image, label in train_dataset.take(3):
    plt.imshow(image.numpy())
    plt.title(f"label : {CLASS_NAMES[label]}")
    plt.show()


2025-08-03 22:48:51.143251: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".
2025-08-03 22:48:51.638217: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: PERMISSION_DENIED: Error executing an HTTP request: HTTP response code 403 with body '<?xml version='1.0' encoding='UTF-8'?><Error><Code>AccessDenied</Code><Message>Access denied.</Message><Details>Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist).</Details></Error>

PermissionDeniedError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error executing an HTTP request: HTTP response code 403 with body '<?xml version='1.0' encoding='UTF-8'?><Error><Code>AccessDenied</Code><Message>Access denied.</Message><Details>Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object. Permission 'storage.objects.get' denied on resource (or it may not exist).</Details></Error>'
	 when reading gs://cloud-ml-data/img/flower_photo/train_set.csv [Op:IteratorGetNext] name: 